In [21]:
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import numpy as np

In [22]:
df=pd.read_csv('movie_data.csv')

In [23]:
df.head(5)

,review,sentiment
0,"The infamous Ed Wood ""classic"" Plan 9 From Out...",0
1,"Sports movies have never been my thing, but a ...",1
2,this movies is really special ! it's about a y...,1
3,"Well, after seeing ""Beginning"" I thought why t...",0
4,"Ben (a fine Charles Bateman), his young daught...",1


In [24]:
def preprocessor(text):
    text = re.sub('<[^>]*>','',text)
    emo = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text = (re.sub('[\W]+','',text.lower())+' '.join(emo).replace('-',''))
    return text

In [25]:
df['review_pro']=df['review'].apply(preprocessor)

In [26]:
df.head(5)

,review,sentiment,review_pro
0,"The infamous Ed Wood ""classic"" Plan 9 From Out...",0,theinfamousedwoodclassicplan9fromouterspacefea...
1,"Sports movies have never been my thing, but a ...",1,sportsmovieshaveneverbeenmythingbutasmallhandf...
2,this movies is really special ! it's about a y...,1,thismoviesisreallyspecialitsaboutayoungfrenchw...
3,"Well, after seeing ""Beginning"" I thought why t...",0,wellafterseeingbeginningithoughtwhythehellthey...
4,"Ben (a fine Charles Bateman), his young daught...",1,benafinecharlesbatemanhisyoungdaughterktacuteg...


In [27]:
#count=CountVectorizer()
#bag =count.fit_tranform(doc)
#print(count.vocabulary_)
#print(bag.toarray())

In [28]:
#Tfid transformer
#tfid = TfidfTransformer(use_idf=True, norm='12',smooth_idf=True)
#np.set_printoptions(precision=2)
#print(TfidfTransformer.transform(count.fit_transform(doc)).toarray())
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:

from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
#nltk.download('stopwords')
#tokenizar usando la raiz de la palabra

def tokenizer(text:str):
    return text.split()

porter = PorterStemmer()
def tokenizer_porte(text: str):
    return [porter.stem(word) for word in text.split()]

stop = stopwords.words('english')
#[w for w in tokenizer(doc) if w not in stop]

In [30]:
X_train = df.loc[:25000, 'review'].values
X_test = df.loc[25000:, 'review'].values
y_train = df.loc[:25000,'sentiment'].values
y_test = df.loc[25000:, 'sentiment'].values

In [31]:
pd.Series(X_test).isnull().sum()


0

In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
tfid = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)
param_grid = [{'vect__ngram_range':[(1,1)],
              'vect__stop_words': [stop,None],
              'vect__tokenizer' : [tokenizer, tokenizer_porte],
              'clf__penalty' : ['l2'],
              'clf__C' : [1.0,10.0,100.0]},
              {'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop,None],
               'vect__tokenizer' : [tokenizer,tokenizer_porte],
               'vect__use_idf': [False],
               'vect__norm' : [None],
               'clf__penalty' : ['l2'],
               'clf__C' : [1.0,10.0,100.0]
               }
              ]

le_tfidf =Pipeline([('vect',tfid),('clf',LogisticRegression(random_state=0,max_iter=500))])

gs_lr_tfidf = GridSearchCV(le_tfidf, param_grid,scoring='accuracy',
                           cv=5,verbose=1,n_jobs=4)

gs_lr_tfidf.fit(X_train, y_train)
print('Best Params',gs_lr_tfidf.best_params_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


c:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Best Params {'clf__C': 100.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x0000029323D074C0>}
